<a href="https://colab.research.google.com/github/TiFcode/TiFai/blob/main/TiFai_tif3_object_recognition_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('*** START ********************************************************')

!pwd
!ls

outputDetails = False
size = (224, 224)
googleDriveFolderPath = '/content/drive/My Drive/Personal/Dev/ChatGPT/2023-01-05 - AI - object recognition/'

In [ ]:
from google.colab.patches import cv2_imshow

def display_image(image, description):
  a=1
  print(f'{description}:')
  print(f'Image size: {image.shape}')
  cv2_imshow(image)

In [ ]:
from google.colab import drive
import glob

# Mount Google Drive
drive.mount('/content/drive')
!pwd
!ls

# Search for a file by name
#pattern = '/content/drive/My Drive/**/*20230105_151957.jpg'
#filenames = glob.glob(pattern, recursive=True)
#print(filenames)


In [ ]:
import os
import cv2

def loadAndProcessAllFilesFromFolder(folderPath):
  filenames = os.listdir(folderPath)

  imagesArray = []
  for filename in filenames:
    filenameFullPath = folderPath + filename
    if outputDetails:
      print(f'Found file [{filename}] with size [{os.stat(filenameFullPath).st_size}] bytes.')
    
    image = cv2.imread(filenameFullPath)
    if outputDetails:
      display_image(image, filenameFullPath)
    imagesArray.append(image)

    if outputDetails:
      print(f'imagesArray contains [{len(imagesArray)}] elements.')

  # Resize the images to a fixed size
  imagesArray = [cv2.resize(image, size) for image in imagesArray]

  # Convert the images to a format that can be used as input to a CNN
  imagesArray = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in imagesArray]

  return imagesArray

In [ ]:
print('Loading object images...')
object_images = loadAndProcessAllFilesFromFolder(googleDriveFolderPath + 'object_images/')
print(f'object_images contains [{len(object_images)}] images.')

print('Loading search images...')
search_images = loadAndProcessAllFilesFromFolder(googleDriveFolderPath + 'search_images/')
print(f'search_images contains [{len(search_images)}] images.')

print('*** FINISH ********************************************************')
